# EPFL Big Data Analysis Week 1 Assignment

## Objective
Produce a rudimentary metric using Wikipedia articles to provide a popularity ranking for programming languages.  
Determine whether this ranking bear any relation to the popular Red Monk ranking.

## Steps
1. The file wikipedia.dat contains full text articles from Wikipedia.  Read in the source data as a distributed dataset
2. Produce a simple metric - this simple metric is a ranking based on the number of articles that mention the language at least once
3. Create an inverted index - maps the language name to the collection of Wikipedia articles mentioned that language

In [1]:
## Helper definitions and classes

langs = ["JavaScript", "Java", "PHP", "Python", "C#", "C++", "Ruby", "CSS",
    "Objective-C", "Perl", "Scala", "Haskell", "MATLAB", "Clojure", "Groovy"]

class WikiArticle:
    def __init__(self, title, text):
        self.title = title
        self.text = text
        
def readWikiArticle(filepath):
    subs = "</title><text>"
    l = len(subs)
    articles = []
    with open(filepath) as f:
         for line in f:
            i = int(line.find(subs))
            title = line[14:i]
            text  = line[i + l: l-16]
            articles.append(WikiArticle(title, text))
    return articles

data = [a.text for a in readWikiArticle('/data/epfl-big-data-analysis/wikipedia.dat')]
print(type(data))

<class 'list'>


In [ ]:
any('Taxobox' in s for s in data[0].split())

In [2]:
## Setup
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("EPFL Wk1 Assignment") \
    .getOrCreate()

# Create an RDD
articles = spark.sparkContext.parallelize(data[:10])
type(articles)


pyspark.rdd.RDD

In [18]:
### Implement the simple metrics

# Count the number of articles that menioned a language at least once
# Implementation 1:
# This implementation will materialise count immediately, 
# instead we prefer to defer the operation since this is not the final results we want
def occurrencesOfLangUsingCount(lang, articlesRDD):
    articlesRDD.filter(lambda a: any(lang in s for s in a.split())).count()

# Implementation 2: Count using aggregate (recommended by course instructor)
# Aggregate is used because we want to return int not the article
def occurencesOfLang(lang, articlesRDD):
    articlesRDD.aggregate(0, lambda acc, a: acc + mentionedLang(lang, a), lambda acc1, acc2: acc1 + acc2)

def mentionedLang(lang, article):
    return any(lang in s for s in article.split())

print(articles.count())
# Test the langSearch function locally
print(mentionedLang('Taxobox', data[0]))

print(articles.filter(lambda a: mentionedLang('Taxobox', a)).count())
print(occurrencesOfLangUsingCount('Taxobox', articles)) # Not working

10
True
1
None


In [ ]:
# Return a list of pairs (k, v) where k is the language 
# and v is the count of articles mentioned that language
# sorted in descending order of count
def rankLangs():
    langs